<a href="https://colab.research.google.com/github/yusica09/seoul-AI-hub-study/blob/main/4%EC%A3%BC%EC%B0%A8/Proj)_Spaceship_Titanic_classification_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spaceship Titanic with DL
### 배경
* 우주의 미스터리를 풀기 위해 데이터 과학 기술이 필요한 2912년에 오신 것을 환영합니다. 4광년 떨어진 곳에서 전송을 받았는데 상태가 좋지 않습니다.

* 우주선 타이타닉은 한 달 전에 발사된 성간 여객선이었습니다. 약 13,000명의 승객을 태운 이 선박은 우리 태양계에서 가까운 별을 도는 새로 거주 가능한 세 개의 외계 행성으로 이민자들을 수송하는 첫 항해를 시작했습니다.

* 첫 번째 목적지인 55 Cancri E로 가는 도중 Alpha Centauri를 돌던 중 부주의한 우주선 Titanic이 먼지 구름 속에 숨겨진 시공간 변칙과 충돌했습니다. 안타깝게도 1000년 전의 이름과 비슷한 운명을 맞이했습니다. 배는 온전했지만 승객의 거의 절반이 다른 차원으로 이동했습니다!

### 데이터 정보
* *PassengerId*
    - 각 승객의 고유 ID. 각 Id는 승객이 함께 여행하는 그룹을 나타내고 그룹 내의 번호를 나타내는 형식을 취합니다 . 그룹의 사람들은 종종 가족 구성원이지만 항상 그런 것은 아닙니다.

* *HomePlanet*
    - 승객이 출발한 행성으로, 일반적으로 승객이 거주하는 행성입니다.

* *CryoSleep*
    - 승객이 항해 기간 동안 냉동 수면 선택했는지 여부를 나타냅니다. cryosleep의 승객은 객실에 갇혀 있습니다.

* *Cabin*
    - 승객이 머무르는 캐빈 번호. 형식을 취합니다 deck/num/side. 여기 에서 Port 또는 Starboard 가 side될 수 있습니다.

* *Destination*
    - 승객이 내릴 행성.

* *Age*
    - 승객의 나이.

* *VIP*
    - 승객이 항해 중 특별 VIP 서비스 비용을 지불했는지 여부.

* *RoomService, FoodCourt, ShoppingMall, Spa, VRDeck*
    - 승객이 Spaceship Titanic 의 다양한 고급 편의 시설 각각에 대해 청구한 금액입니다.

* *Name*
    - 승객의 성과 이름.

* *Transported*
    - 승객이 다른 차원으로 이동했는지 여부. 정답 데이터입니다.

## import library

In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data Load
### Read CSV files wit pandas

In [3]:
train_data = pd.read_csv("/content/drive/MyDrive/Datasets/spaceship_titanic_train_data.csv")
train_labels = pd.read_csv("/content/drive/MyDrive/Datasets/spaceship_titanic_train_labels.csv")

test_data = pd.read_csv("/content/drive/MyDrive/Datasets/spaceship_titanic_test_data.csv")
test_labels = pd.read_csv("/content/drive/MyDrive/Datasets/spaceship_titanic_test_labels.csv")

train = pd.concat([train_data, train_labels], axis=1)
test = pd.concat([test_data, test_labels], axis=1)

### Preprocessing
* 결측치 제거 후 데이터 로더에 연결

In [4]:
train = train.fillna(method='bfill')
test = test.fillna(method='bfill')

In [5]:
train.dtypes

PassengerId      object
HomePlanet       object
CryoSleep          bool
Cabin            object
Destination      object
Age             float64
VIP                bool
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [6]:
# a = train.sort_values(by='PassengerId')
# a

In [7]:
# train['PassengerId'] = train['PassengerId'].apply(lambda x: x.split('_')[0])
# train['PassengerId'] = train['PassengerId'].astype(int)
# train

In [8]:
# 일부 dtype은 tensor로 변경 불가
train['HomePlanet'] = train['HomePlanet'].astype('category')
train['CryoSleep'] = train['CryoSleep'].map({True: 1, False: 0})
train['VIP'] = train['VIP'].map({True: 1, False: 0})
train['Transported'] = train['Transported'].map({True: 1, False: 0})

test['HomePlanet'] = test['HomePlanet'].astype('category')
test['CryoSleep'] = test['CryoSleep'].map({True: 1, False: 0})
test['VIP'] = test['VIP'].map({True: 1, False: 0})
test['Transported'] = test['Transported'].map({True: 1, False: 0})

In [9]:
train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,2513_01,Earth,0,F/575/P,TRAPPIST-1e,28.0,0,0.0,55.0,0.0,656.0,0.0,Loree Mathison,0
1,2774_02,Earth,0,F/575/P,TRAPPIST-1e,17.0,0,0.0,1195.0,31.0,0.0,0.0,Crisey Mcbriddley,0
2,8862_04,Europa,1,C/329/S,55 Cancri e,28.0,0,0.0,0.0,0.0,0.0,0.0,Alramix Myling,1
3,8736_02,Mars,0,F/1800/P,TRAPPIST-1e,20.0,0,0.0,2.0,289.0,976.0,0.0,Tros Pota,1
4,0539_02,Europa,1,C/18/P,55 Cancri e,36.0,0,0.0,0.0,0.0,0.0,0.0,Achyon Nalanet,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6949,6076_01,Earth,0,G/988/S,TRAPPIST-1e,18.0,0,14.0,2.0,144.0,610.0,0.0,Therry Cames,1
6950,5537_01,Mars,0,F/1063/S,TRAPPIST-1e,50.0,0,690.0,0.0,30.0,762.0,428.0,Herms Bancy,0
6951,5756_06,Earth,0,F/1194/P,PSO J318.5-22,22.0,0,158.0,0.0,476.0,0.0,26.0,Karena Briggston,0
6952,0925_01,Mars,0,F/191/P,TRAPPIST-1e,34.0,0,379.0,0.0,1626.0,0.0,0.0,Skix Kraie,0


In [10]:
test['PassengerId'] = test['PassengerId'].apply(lambda x: x.split('_')[0])
test['PassengerId'] = test['PassengerId'].astype(int)

### Data Loader

In [11]:
batch_size = 4

def df_to_dataset(dataframe, label_name="Transported", shuffle=True, batch_size=batch_size):
    dataframe = dataframe.copy()
    labels = dataframe.pop(label_name)
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
        ds = ds.repeat()
    ds = ds.batch(batch_size)

    return ds


In [12]:
train_ds = df_to_dataset(train)
train_ds

<_BatchDataset element_spec=({'PassengerId': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'HomePlanet': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'CryoSleep': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Cabin': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Destination': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Age': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'VIP': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'RoomService': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'FoodCourt': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'ShoppingMall': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Spa': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'VRDeck': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Name': TensorSpec(shape=(None,), dtype=tf.string, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [13]:
test_ds = df_to_dataset(test, shuffle=False)
test_ds

<_BatchDataset element_spec=({'PassengerId': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'HomePlanet': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'CryoSleep': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Cabin': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Destination': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'Age': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'VIP': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'RoomService': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'FoodCourt': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'ShoppingMall': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Spa': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'VRDeck': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'Name': TensorSpec(shape=(None,), dtype=tf.string, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [14]:
for t, l in train_ds:
  print(t, l)
  break

for t, l in test_ds:
  print(t, l)
  break


{'PassengerId': <tf.Tensor: shape=(4,), dtype=string, numpy=array([b'0128_01', b'0408_01', b'4866_01', b'5128_02'], dtype=object)>, 'HomePlanet': <tf.Tensor: shape=(4,), dtype=string, numpy=array([b'Mars', b'Mars', b'Earth', b'Mars'], dtype=object)>, 'CryoSleep': <tf.Tensor: shape=(4,), dtype=int64, numpy=array([0, 0, 0, 0])>, 'Cabin': <tf.Tensor: shape=(4,), dtype=string, numpy=array([b'D/3/S', b'E/25/P', b'F/922/S', b'F/1045/P'], dtype=object)>, 'Destination': <tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'TRAPPIST-1e', b'TRAPPIST-1e', b'TRAPPIST-1e', b'TRAPPIST-1e'],
      dtype=object)>, 'Age': <tf.Tensor: shape=(4,), dtype=float64, numpy=array([61., 35., 17., 21.])>, 'VIP': <tf.Tensor: shape=(4,), dtype=int64, numpy=array([1, 0, 0, 0])>, 'RoomService': <tf.Tensor: shape=(4,), dtype=float64, numpy=array([2353., 1134.,    0.,    0.])>, 'FoodCourt': <tf.Tensor: shape=(4,), dtype=float64, numpy=array([334., 113.,   0.,   1.])>, 'ShoppingMall': <tf.Tensor: shape=(4,), dtype=floa

In [15]:
list(set(train['VIP']))

[0, 1]

## Preprocessing with layers

In [16]:
inputs = {
  'PassengerId': tf.keras.Input(shape=(), dtype='string'),
  'CryoSleep': tf.keras.Input(shape=(), dtype='int64'),
  'HomePlanet': tf.keras.Input(shape=(), dtype='string'),
  'RoomService': tf.keras.Input(shape=(), dtype='float64'),
  'VIP': tf.keras.Input(shape=(), dtype='string'),
  'Cabin': tf.keras.Input(shape=(), dtype='string')
}

split_text = tf.strings.split(inputs['Cabin'], sep="/")

num_text = tf.strings.split(inputs['PassengerId'], sep="_")
id_num = tf.strings.to_number(num_text, out_type=tf.int64)

# Convert index to one-hot; e.g. [2] -> [0,1].
type_output = tf.keras.layers.CategoryEncoding(num_tokens=2, output_mode='one_hot')(inputs['CryoSleep'])
print(type_output.shape)
dense_type = tf.keras.layers.Dense(2, activation='relu')(type_output)
print(dense_type.shape)

vip = tf.keras.layers.StringLookup(vocabulary=["True", "False"], num_oov_indices=0, output_mode='one_hot')(inputs['VIP'])
print(vip.shape)

# Convert size strings to indices; e.g. ['small'] -> [1].
size_output = tf.keras.layers.StringLookup(vocabulary=list(set(train['HomePlanet'])))(inputs['HomePlanet'])
size_output = tf.keras.layers.Reshape([-1])(size_output)
print(size_output.shape)
dense_size = tf.keras.layers.Dense(2, activation='relu')(size_output)
print(dense_size.shape)

# Normalize the numeric inputs; e.g. [2.0] -> [0.0].
weight_output = tf.keras.layers.Normalization(
      axis=None, mean=2.0, variance=1.0)(inputs['RoomService'])
weight_output = tf.keras.layers.Reshape([-1])(weight_output)
print(weight_output.shape)
dense_weight = tf.keras.layers.Dense(2, activation='relu')(weight_output)
print(dense_weight.shape)

x = tf.concat([dense_type, dense_size, dense_weight], -1) # batch, 특징 (여기로 합쳐라)
x = tf.keras.layers.Reshape([-1, 1])(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(1024, return_sequences=False))(x)

outputs = tf.keras.layers.Dense(1)(x) # Sigmoid, BCE loss

# outputs = {
#   'PassengerId': id_num,
#   'CryoSleep': type_output,
#   'HomePlanet': size_output,
#   'RoomService': weight_output,
#   # 'VIP':vip,
#   's': split_text
# }

preprocessing_model = tf.keras.Model(inputs, outputs)

(None, 2)
(None, 2)
(None, 2)
(None, 1)
(None, 2)


/usr/local/lib/python3.10/dist-packages/numpy/core/numeric.py:2463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


(None, 1)
(None, 2)


## Input and preprocessing Layers

### String Look Up


In [17]:
 tf.strings.split(train['Cabin'], "/")[:,-1:].numpy()

array([[b'P'],
       [b'P'],
       [b'S'],
       ...,
       [b'P'],
       [b'P'],
       [b'P']], dtype=object)

In [18]:
inputs = {
    'Cabin': tf.keras.Input(shape=(), dtype='string')
}

# 캐빈 데이터를 분할합니다
cabin_split = tf.strings.split(inputs['Cabin'], "/")

# 마지막 요소만 선택합니다
cabin_last = cabin_split.to_tensor()[:, -1]

# StringLookup 레이어를 사용하여 one-hot 인코딩을 수행합니다
cabin_output = tf.keras.layers.StringLookup(vocabulary=["S", "P"], num_oov_indices=1, output_mode='one_hot')(cabin_last)

outputs = {
    'Cabin': cabin_output
}

model = tf.keras.Model(inputs, outputs)

In [19]:
for t, l in train_ds:
    pred = model(t)
    print(pred['Cabin'])
    break

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['PassengerId', 'HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


tf.Tensor(
[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]], shape=(4, 3), dtype=float32)


In [20]:
s = "F/575/P"
tf.strings.split(s, sep="/")

<tf.Tensor: shape=(3,), dtype=string, numpy=array([b'F', b'575', b'P'], dtype=object)>

In [21]:
list(set(train['HomePlanet']))

['Earth', 'Europa', 'Mars']

In [22]:
string_lookup_layer = tf.keras.layers.StringLookup(
    vocabulary=list(set(train['HomePlanet'])),
    num_oov_indices=0,
    output_mode='one_hot')
"""
int 0, 1, 2 (idx)
one_hot [1, 0, 0]
multi_hot [1, 0, 1] (다중입력)
"""

string_lookup_layer(list(train['HomePlanet'].to_numpy()))

<tf.Tensor: shape=(6954, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)>

### Category Encoding

In [23]:
train['CryoSleep']

0       0
1       0
2       1
3       0
4       1
       ..
6949    0
6950    0
6951    0
6952    0
6953    0
Name: CryoSleep, Length: 6954, dtype: int64

In [24]:
one_hot_layer = tf.keras.layers.CategoryEncoding(
    num_tokens=2, output_mode='one_hot')
one_hot_layer(list(train['CryoSleep'].to_numpy()))

<tf.Tensor: shape=(6954, 2), dtype=float32, numpy=
array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)>

### Normalization

In [25]:
normalization_layer = tf.keras.layers.Normalization(mean=1.0, variance=10.0)

# 각 입력 값에서 mean 값을 뺍니다. 예를 들어, 입력이 [1., 2., 3.]라면, 결과는 [-2., -1., 0.]이 됩니다.
# 다음으로, 이 결과를 variance의 제곱근 값, 즉 sqrt(2.)로 나눕니다. 따라서 결과는 [-2/sqrt(2), -1/sqrt(2), 0]가 됩니다. (정규화 과정)

print(normalization_layer(train['RoomService'][:5]).numpy())
print(train['RoomService'][:5].to_numpy())

[-0.31622776 -0.31622776 -0.31622776 -0.31622776 -0.31622776]
[0. 0. 0. 0. 0.]


### Model Train

In [26]:
preprocessing_model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [28]:
max_epochs = 20

history = preprocessing_model.fit(train_ds,
                                  epochs=max_epochs,
                                  steps_per_epoch=len(train) // batch_size,
                                  validation_data=test_ds,
                                  validation_steps=len(test) // batch_size)

Epoch 1/20
1738/1738 [==============================] - 22s 12ms/step - loss: 0.5490 - accuracy: 0.7169 - val_loss: 0.5471 - val_accuracy: 0.7241
Epoch 2/20
1738/1738 [==============================] - 19s 11ms/step - loss: 0.5459 - accuracy: 0.7146 - val_loss: 0.5439 - val_accuracy: 0.7241
Epoch 3/20
1738/1738 [==============================] - 21s 12ms/step - loss: 0.5464 - accuracy: 0.7179 - val_loss: 0.5408 - val_accuracy: 0.7281
Epoch 4/20
1738/1738 [==============================] - 20s 12ms/step - loss: 0.5433 - accuracy: 0.7152 - val_loss: 0.5421 - val_accuracy: 0.7281
Epoch 5/20
1738/1738 [==============================] - 18s 11ms/step - loss: 0.5434 - accuracy: 0.7126 - val_loss: 0.5391 - val_accuracy: 0.7241
Epoch 6/20
1738/1738 [==============================] - 19s 11ms/step - loss: 0.5416 - accuracy: 0.7152 - val_loss: 0.5404 - val_accuracy: 0.6745
Epoch 7/20
1738/1738 [==============================] - 19s 11ms/step - loss: 0.5411 - accuracy: 0.7122 - val_loss: 0.5391 -